Speak to any Kaggle Competitor: Histogram Gradient Boosting is 'the model' for tabular data. The fact that Histogram Gradient Boosting is so fast, so robust and so dependible is actually astonishing and difficult to ignore. The fact that these approaches are also so explainable makes them a 'must have' in feature engineering and has even gotten [bankers excited](https://www.youtube.com/watch?v=RLxR5fqVQm0&list=WL&index=11&t=1738s). What makes these methods so good is just how robust they are to garbage: data which is irrelevant, multicollinear, unscaled and of mixed type, as well as how portable they have become across languages and applictions. There is good [theory why these methods work so well](https://www.youtube.com/watch?v=wPqtzj5VZus&list=WL&index=8&t=2164s), related to VC theory and the orthogonality of the predictions of the individual trees. 
  
However, there are still opppotunities and area where trees struggle:
1. trees can have poor performance in multi-output problems, as you need to fit a model for each individual class and cannot benefit from shared representations
2. trees cannot compute gradients over state-of-the-art image recognition backbones to incorperate image, pretrained embeddings or audio data, and 
3. trees cannot be used with self-supervised pretraining in cases where you may have few labels
That's what [TabNet](https://arxiv.org/pdf/1908.07442.pdf.) promises: robustness and explainability on tabular data without losing out on the tricks and advantages which Deep Learning approaches can offer in certain contexts. 
  
![tabnet](https://github.com/titu1994/tf-TabNet/raw/master/images/TabNet.png?raw=true)
The magic behind TabNets comparible robustness and explainability relies on its use of transformers. Transformers have become an increasingly popular and powerful architecture in Natural Language and even Image data, and now are finding use in Tabular Data as well. [Transformers are attention mechanisms](https://www.youtube.com/watch?v=iDulhoQ2pro) which serve to mask certain features at different stages of the model. By masking these features, TabNet is able to indicate what features are most used globally, similar to how tree ensembles describe global feature importance, and local importance or attribution as well. Tabnet also draws on a number of other tricks from object detection problems by using [residual connections](https://www.youtube.com/watch?v=GWt6Fu05voI), along with [batch normalization](https://www.youtube.com/watch?v=nUUqwaxLnWs).  I've tagged great resources here as these concepts need a lot of time on their own and are outside the scope of this notebook. 
  
TabNet has two useful implemnetations in [Pytorch](https://github.com/dreamquark-ai/tabnet) and, by the original authors, in [Tensorflow](https://github.com/google-research/google-research/tree/master/tabnet).  In this notebook we will look at the Pytorch implementation which has more community support and benchmarking, and compare our results under default parameters to a Feed-forward Dense Neural Network provided in Scikit-learn, and CatBoost. We will compare the global feature importances described by TabNet and our Catboost and let readers get a sense of the usability and promise of these tools. 

# Data
We will be looking at a customer churn classification problem for broadband internet customers. The aim of this notebook is not to explore many complicated approaches to featur engineering but to see how mature and dependible these different model classes are in 'out-the-box' performance. The main aim in choosing a dataset was for it to be reasonably large, at around 510125 observations, and to have mixed categorical, count and continuous data, as it common to tabular datasets. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
CATEGORICAL_COLUMNS = ['line_stat', 'serv_type', 'serv_code',
                       'bandwidth', 'term_reas_code', 'term_reas_desc',
                       'with_phone_service', 'current_mth_churn']
NUMERIC_COLUMNS = ['contract_month', 'ce_expiry', 'secured_revenue', 'complaint_cnt']

df = pd.read_csv('/kaggle/input/broadband-customers-base-churn-analysis/bbs_cust_base_scfy_20200210.csv').assign(complaint_cnt = lambda df: pd.to_numeric(df.complaint_cnt, 'coerce'))
df.loc[:, NUMERIC_COLUMNS] = df.loc[:, NUMERIC_COLUMNS].astype(np.float32).pipe(lambda df: df.fillna(df.mean())).pipe(lambda df: (df - df.mean())/df.std())
df.loc[:, CATEGORICAL_COLUMNS] = df.loc[:, CATEGORICAL_COLUMNS].astype(str).applymap(str).fillna('')
df = df.groupby('churn').apply(lambda df: df.sample(df.churn.value_counts().min()))
df.head()

We will be taking a simple randomized test-train split approach to cross-validation, though in other applications k-fold, stratified k-fold or backtesting may be more appropriate. 

In [ ]:
from sklearn.model_selection import train_test_split

def get_labels(x: pd.Series) -> pd.Series:
    """
    Converts strings to unqiue ints for use in Pytorch Embedding
    """
    labels, levels = pd.factorize(x)
    return pd.Series(labels, name=x.name, index=x.index)

X, E, y = (df
           .loc[:, NUMERIC_COLUMNS]
           .astype('float32')
           .join(pd.get_dummies(df.loc[:, CATEGORICAL_COLUMNS])),
           df
           .loc[:, NUMERIC_COLUMNS]
           .astype('float32')
           .join(df.loc[:, CATEGORICAL_COLUMNS].apply(get_labels)),
           df.churn == 'Y')

X_train, X_valid, E_train, E_valid, y_train, y_valid = train_test_split(X.to_numpy(), E.to_numpy(), y.to_numpy())

# Tabnet

We ended up limiting the the number of epochs for the model, so that even on GPU, the time taken for training was roughly equivalent. This makes our comparison more faithful by considering a computational budget for training. Unlike our other models we rely on 1 dimensional embedding layers for representing our categorical data, as opposed to a one-hot encoding. 

In [ ]:
! pip install --quiet pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score

cat_idxs = [E.columns.get_loc(c) for c in E.select_dtypes(exclude='float32').columns]

clf = TabNetClassifier(cat_idxs=cat_idxs, cat_emb_dim=1)
clf.fit(E_train, y_train, E_valid, y_valid, max_epochs=15)
preds = clf.predict(E_valid)

tabnet_accuracy = accuracy_score(y_valid, preds)
tabnet_accuracy

By looking at out Attention layers we can get a sense of global feature importance across the model. Here we see `ce_expiry` is the most important feature, followed by `with_phone_service`. This can be useful in validating the usefulness of engineered features and guiding domain experts. 

In [ ]:
pd.Series(clf.feature_importances_, index=E.columns).plot.bar(title=f'TabNet Global Feature Importances ({round(tabnet_accuracy*100, 2)}% Accuracy)')

Using the attention masks you can also observe local attributions across the network, useful in applications such as credit scoring where customers demand insight into the scores given to them. 

In [ ]:
import matplotlib.pyplot as plt
explain_matrix, masks = clf.explain(E_valid)

fig, axs = plt.subplots(1, 3, figsize=(20,20))

for i in range(3):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")

We can see even for this simple case the model ends up being quite large in the number of learnable parameter, which may make it less suitible for certain datasets or in certain edge applications. 

In [ ]:
print(clf.network)

You can see that with a large number of features in our model's embedding layer, TabNet can have many parameters to train making it inappropriate in contexts where data is limited. 

In [ ]:
tabnet_parameters = sum([param.nelement() for param in clf.network.parameters()])
print(f'# parameters: {tabnet_parameters}')


# Comparison: CatBoost

In recent years CatBoost has garnered a lot of attention based on its novel and robust approach to [handling sparse categorical data and obvious speeed](https://catboost.ai/#benchmark). Compared to similar histogram gradient boosting approaches offere by LightGBM and XGBoost, Catboost prioritized symetrical trees and experiments with merging categorical data.  It has become a popular 'out-the-box' model for Kaggle competitors with a rich excosystem of support and extensions build around it. 

In [ ]:
from catboost import CatBoostClassifier

C_train = pd.DataFrame(E_train, columns=E.columns)
C_train.loc[:, CATEGORICAL_COLUMNS] = C_train.loc[:, CATEGORICAL_COLUMNS].astype(str)

Here, we stick to the proven default parameters offered by Yandex for Catboost which owe to ease of use across contexts. 

In [ ]:
%time
cat = CatBoostClassifier(verbose=0, 
                         task_type="GPU")
cat.fit(E_train, y_train)

In [ ]:
C_valid = pd.DataFrame(E_valid, columns=E.columns)
C_valid.loc[:, CATEGORICAL_COLUMNS] = C_valid.loc[:, CATEGORICAL_COLUMNS].astype(str)

Reporting on the accuracy we still see Catboost outperforming TabNet on this single class problem. In the TabNet paper, authors benchmark against multiclass problems where shared representations present a clear advantage over Multioutput Ensemble Models where we train our models in a one-vs-the-rest fashion. 

In [ ]:
cat_pred = cat.predict(C_valid) == 'True'
catboost_accuracy = accuracy_score(y_valid, cat_pred)
catboost_accuracy

What is interesting is how we compare the feature importances learn from either approach.  With experimentation the features importances learned by the Ensemble Model seemed far more consistent across refits, training steps or different hyperparameters than TabNet, but in many cases presented fairly similar results. 

In [ ]:
pd.Series(cat.feature_importances_, index=E.columns).plot.bar(title=f'Catboost Global Feature Importances ({round(catboost_accuracy*100, 2)}% Accuracy)')

# Comparison: MLP

Lastly, we will look and compare our TabNet to a similar Multi-layer Perceptron (MLP). Finding a apples-to-apples comparison here is challenging as MLP's can have an array of architectures, activation functions, learning rates, initializers, optimizers and restarts which may affect how the model converges.  Here I looke to compare TabNet to a MLP in two ways: first, with a single hidden layer, and second, with three hidden layers. In all cases I tried to keep the total number of parameters similar to TabNet, so we could attribute performance to archetecture rather than perhaps capacity or size. 

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Shallow model

In [ ]:
# lazy line search
for h in range(25):
    equivalent_parameters = (X.shape[1] * h  + h) + (h + 1)
    if  equivalent_parameters >= tabnet_parameters:
        break
h

In [ ]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('mlp', MLPClassifier(hidden_layer_sizes=(h, )))])
pipeline.fit(X_train, y_train)

In [ ]:
sum([np.sum(l**0) for l in pipeline.named_steps['mlp'].coefs_ ] + [np.sum(l**0) for l in pipeline.named_steps['mlp'].intercepts_ ] )

In [ ]:
mlp_y_pred = pipeline.predict(X_valid)
accuracy_score(y_true=y_valid, y_pred=mlp_y_pred)

Deeper model

In [ ]:
# lazy line search
for j in range(25):
    equivalent_parameters = (X.shape[1] * j  + j) + (j*j + j) + (j*j + j) + (j + 1)
    if  equivalent_parameters >= tabnet_parameters:
        break
j

In [ ]:
pipeline = Pipeline([('scale', StandardScaler()), 
                     ('mlp', MLPClassifier(hidden_layer_sizes=(j, j, j, )))])
pipeline.fit(X_train, y_train)

In [ ]:
accuracy_score(y_true=y_valid, y_pred=mlp_y_pred)

In [ ]:
sum([np.sum(l**0) for l in pipeline.named_steps['mlp'].coefs_ ] + [np.sum(l**0) for l in pipeline.named_steps['mlp'].intercepts_ ] )

Across both MLPs we see considerably worse performance than TabNet and our Ensemble method. While explainations and features importances can be garnered from the model using an integrated gradient, DeepLift or saliency based approaches, I don't think these methods offer the elegance of TabNet in this regard, which I feel makes it a god fit in tabular data problems. 

# Conclusion
I am really excited about TabNet, I think with further development we have a great candidate for modelling Tabular Data in certain scenarios. I think for now those scenarios will remain multi-output problems, where we can benefit from online-learning and embeddings across large datasets. I think other Attention-Based approaches to modelling tabular data are starting to take off such as Soft Decision Trees (which I have written about and implemented before) and [Neural Oblivious Decision Ensembles](https://arxiv.org/abs/1909.06312). Let's see where things go. 